In [1]:
# union all 1x2_odds files together and join to master data


In [2]:
# imports
import pandas as pd

In [3]:
# get 1x2 odds data for last few years
games_0 = pd.read_excel('data/2023_1x2_data.xlsx', header=0)
games_1 = pd.read_excel('data/2024_1x2_data.xlsx', header=0)
games_2 = pd.read_excel('data/2025_1x2_data.xlsx', header=0)

# union together
all_1x2_odds = pd.concat([games_0, games_1, games_2], ignore_index=True)

# reset index
all_1x2_odds.reset_index(drop=True, inplace=True)

# only keep key cols
key_cols = ['Game_ID'] + [x for x in all_1x2_odds.columns if x.startswith('Odds')]
all_1x2_odds = all_1x2_odds[key_cols]

# inspect
all_1x2_odds.info()
all_1x2_odds.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Game_ID  3349 non-null   object 
 1   Odds_1   3224 non-null   float64
 2   Odds_X   3224 non-null   float64
 3   Odds_2   3224 non-null   float64
dtypes: float64(3), object(1)
memory usage: 104.8+ KB


,Game_ID,Odds_1,Odds_X,Odds_2
0,2023-06-24-20:00|Florida Panthers vs Edmonton ...,2.45,4.0,2.45
1,2023-06-21-20:00|Edmonton Oilers vs Florida Pa...,2.35,4.0,2.55
2,2023-06-18-20:00|Florida Panthers vs Edmonton ...,2.00,4.0,3.10
3,2023-06-15-20:00|Edmonton Oilers vs Florida Pa...,2.30,4.0,2.55
4,2023-06-13-20:00|Edmonton Oilers vs Florida Pa...,2.15,4.0,2.80


In [4]:
# read in master games data
all_games = pd.read_excel(r'data/master_games_data.xlsx', header=0)

# inspect
all_games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3349 non-null   object        
 1   Extra_Time                3349 non-null   object        
 2   Home_Team                 3349 non-null   object        
 3   Away_Team                 3349 non-null   object        
 4   Home_Score                3349 non-null   int64         
 5   Away_Score                3349 non-null   int64         
 6   P1_Home_Score             3349 non-null   int64         
 7   P1_Away_Score             3349 non-null   int64         
 8   P2_Home_Score             3349 non-null   int64         
 9   P2_Away_Score             3349 non-null   int64         
 10  P3_Home_Score             3349 non-null   int64         
 11  P3_Away_Score             3349 non-null   int64         
 12  P4_Home_Score       

In [5]:
# join the 2 dfs together
master_data = pd.merge(all_games, all_1x2_odds, how='left', on='Game_ID')

# inspect
master_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3349 entries, 0 to 3348
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3349 non-null   object        
 1   Extra_Time                3349 non-null   object        
 2   Home_Team                 3349 non-null   object        
 3   Away_Team                 3349 non-null   object        
 4   Home_Score                3349 non-null   int64         
 5   Away_Score                3349 non-null   int64         
 6   P1_Home_Score             3349 non-null   int64         
 7   P1_Away_Score             3349 non-null   int64         
 8   P2_Home_Score             3349 non-null   int64         
 9   P2_Away_Score             3349 non-null   int64         
 10  P3_Home_Score             3349 non-null   int64         
 11  P3_Away_Score             3349 non-null   int64         
 12  P4_Home_Score       

In [6]:
# function that takes the df, sorts it by the target grain, then performs the groupby
def grain_sort_cumsum_props(home_vs_away: str, df: pd.DataFrame) -> pd.DataFrame:
    # add home or away team to sort grain
    home_plus_year = [home_vs_away, 'Year']
    
    # combine to create final sort grain
    sort_grain = home_plus_year + ['Date', 'Game_ID'] 

    # sort
    df = df.sort_values(by=sort_grain)

    # hard code cumsum cols
    cum_cols = ["Reg_Home_Win", "Reg_Away_Win", "Reg_Tie", 'P1_Home_Score', 'P1_Away_Score', 
                'P2_Home_Score', 'P2_Away_Score', 'P3_Home_Score', 'P3_Away_Score',
                'regular_time_score_home', 'regular_time_score_away']
    
    # add count of games for home or away
    df[f'cum_{home_vs_away}_Games'] = df.groupby(home_plus_year).cumcount() + 1

    # compute cumsum for each group
    df[[f"cum_{col}" for col in cum_cols]] = df.groupby(home_plus_year)[cum_cols].cumsum()

    # calc prop home reg win
    df['prop_Reg_Home_Win'] = df['cum_Reg_Home_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Away_Win'] = df['cum_Reg_Away_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Tie'] = df['cum_Reg_Tie'] / df[f'cum_{home_vs_away}_Games']

    # calc props for what % of goals scored in each period
    df['prop_P1_Home_Score'] = df['cum_P1_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P2_Home_Score'] = df['cum_P2_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P3_Home_Score'] = df['cum_P3_Home_Score'] / df['cum_regular_time_score_home']

    df['prop_P1_Away_Score'] = df['cum_P1_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P2_Away_Score'] = df['cum_P2_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P3_Away_Score'] = df['cum_P3_Away_Score'] / df['cum_regular_time_score_away']

    # calc goal diffs at the period level
    df['prop_P1_Home_Goal_Diff'] = df['cum_P1_Home_Score'] / df['cum_P1_Away_Score']
    df['prop_P2_Home_Goal_Diff'] = df['cum_P2_Home_Score'] / df['cum_P2_Away_Score']
    df['prop_P3_Home_Goal_Diff'] = df['cum_P3_Home_Score'] / df['cum_P3_Away_Score']

    df['prop_P1_Away_Goal_Diff'] = df['cum_P1_Away_Score'] / df['cum_P1_Home_Score']
    df['prop_P2_Away_Goal_Diff'] = df['cum_P2_Away_Score'] / df['cum_P2_Home_Score']
    df['prop_P3_Away_Goal_Diff'] = df['cum_P3_Away_Score'] / df['cum_P3_Home_Score']

    # calc props for goals scored / allowed
    df['prop_reg_home_goal_diff'] = df['cum_regular_time_score_home'] / (df['cum_regular_time_score_home'] + df['cum_regular_time_score_away'])
    df['prop_reg_away_goal_diff'] = 1 - df['prop_reg_home_goal_diff']
    
    return df

# test func
test_all_home_games = grain_sort_cumsum_props('Home_Team', master_data)

# isolate cumcols
sel_cum_cols = [x for x in test_all_home_games.columns if x.startswith('cum_') or x.startswith('prop_')]

# add date, gid, and home team to list
peek_cols = ['Date', 'Game_ID', 'Home_Team'] + sel_cum_cols

# inspect cum cols
test_all_home_games[peek_cols].tail(5)

,Date,Game_ID,Home_Team,cum_Home_Team_Games,cum_Reg_Home_Win,cum_Reg_Away_Win,cum_Reg_Tie,cum_P1_Home_Score,cum_P1_Away_Score,cum_P2_Home_Score,...,prop_P2_Away_Score,prop_P3_Away_Score,prop_P1_Home_Goal_Diff,prop_P2_Home_Goal_Diff,prop_P3_Home_Goal_Diff,prop_P1_Away_Goal_Diff,prop_P2_Away_Goal_Diff,prop_P3_Away_Goal_Diff,prop_reg_home_goal_diff,prop_reg_away_goal_diff
3128,2025-10-23,2025-10-23-20:00|Winnipeg Jets vs Seattle Kraken,Winnipeg Jets,7,3,3,1,9,3,3,...,0.333333,0.500000,3.00,0.500000,0.777778,0.333333,2.000000,1.285714,0.513514,0.486486
3119,2025-10-24,2025-10-24-20:00|Winnipeg Jets vs Calgary Flames,Winnipeg Jets,8,4,3,1,9,4,7,...,0.333333,0.476190,2.25,1.000000,0.800000,0.444444,1.000000,1.250000,0.533333,0.466667
3103,2025-10-26,2025-10-26-18:00|Winnipeg Jets vs Utah Mammoth,Winnipeg Jets,9,4,4,1,9,4,9,...,0.375000,0.458333,2.25,1.000000,0.727273,0.444444,1.000000,1.375000,0.520000,0.480000
3073,2025-10-30,2025-10-30-20:00|Winnipeg Jets vs Chicago Blac...,Winnipeg Jets,10,5,4,1,11,5,11,...,0.333333,0.481481,2.20,1.222222,0.769231,0.454545,0.818182,1.300000,0.542373,0.457627
3063,2025-11-01,2025-11-01-15:00|Winnipeg Jets vs Pittsburgh P...,Winnipeg Jets,11,6,4,1,13,5,13,...,0.344828,0.482759,2.60,1.300000,0.785714,0.384615,0.769231,1.272727,0.560606,0.439394


In [7]:
# TODO - perform EDA on records that resulted in a regular time tie and see if there's any patterns or trends in this subset that could be indicative of predictive features
master_data[master_data['Reg_Tie'] == True].describe()

,Home_Score,Away_Score,P1_Home_Score,P1_Away_Score,P2_Home_Score,P2_Away_Score,P3_Home_Score,P3_Away_Score,P4_Home_Score,P4_Away_Score,...,Year,Total_Score,regular_time_score_home,regular_time_score_away,regular_time_score_total,Week_of_Year,Game_Start_Hour,Odds_1,Odds_X,Odds_2
count,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,...,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,678.000000,678.000000,678.000000
mean,3.143059,3.072238,0.805949,0.814448,0.932011,0.909348,0.869688,0.883853,0.388102,0.330028,...,2023.679887,6.215297,2.607649,2.607649,5.215297,28.651558,19.209632,2.324646,4.291512,2.900258
std,1.244877,1.280135,0.825494,0.829161,0.882241,0.873729,0.863780,0.847588,0.487663,0.470556,...,0.679648,2.319597,1.159799,1.159799,2.319597,17.574251,2.251863,0.666943,0.293217,0.923385
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2023.000000,1.000000,0.000000,0.000000,0.000000,1.000000,8.000000,1.290000,3.800000,1.380000
25%,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2023.000000,5.000000,2.000000,2.000000,4.000000,11.000000,19.000000,1.842500,4.100000,2.250000
50%,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,2024.000000,7.000000,3.000000,3.000000,6.000000,38.000000,19.000000,2.182500,4.225000,2.700000
75%,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2024.000000,7.000000,3.000000,3.000000,6.000000,44.000000,21.000000,2.625000,4.425000,3.300000
max,8.000000,7.000000,5.000000,4.000000,4.000000,4.000000,5.000000,5.000000,1.000000,1.000000,...,2025.000000,15.000000,7.000000,7.000000,14.000000,52.000000,22.000000,6.125000,6.000000,8.500000


In [ ]:
# TODO - think about what features could be indicative/relevent for identifying matchups that would require some free hockey to settle things

# the team's conf, div, and/or overall rank; in order to calc these ranks need to calc the number of wins at that grain; another way to think about it is total goals scored vs allowed

In [ ]:
# TODO - create cumulative teams table where each row is at the team + date level and I get cumulative:
# need to agg @ weekly level and calc weekly level ranks for simplicity vs daily ranks
# conf wins/losses, division wins/losses, conf rank, div rank
